# Pystata example:
### Using python to call Stata to run regression(s) and report results

In [1]:
## 1. generate random sample of data
import numpy as np
import pandas as pd

# one dependent variables with four indenpendent variables
data1 = pd.DataFrame(np.random.rand(1000,5), columns = ['y','x1','x2','x3','x4'])
# some potential fixed effects or clusters
data1['fx1'] = np.random.randint(10, size=1000)
data1['fx2'] = np.random.randint(5, size=1000)
data1['fx3'] = np.random.randint(3, size=1000)

# generate another data
data2 = pd.DataFrame(np.random.rand(10000,5), columns = ['y','x1','x2','x3','x4'])
# add some modifications
data2['x1'] = data2['x1']*15
data2['x2'] = data2['x2'] + 1
data2['x3'] = np.log(data2['x3']** + 1)


# some potential fixed effects or clusters
data2['fx1'] = np.random.randint(20, size=10000)
data2['fx2'] = np.random.randint(15, size=10000)
data2['fx3'] = np.random.randint(13, size=10000)


### Dataset 1

In [2]:
data1.head()

,y,x1,x2,x3,x4,fx1,fx2,fx3
0,0.800794,0.303683,0.249580,0.072311,0.135722,7,3,2
1,0.979322,0.361632,0.548884,0.136269,0.937502,9,0,2
2,0.805480,0.750954,0.333014,0.958121,0.129445,2,2,2
3,0.797144,0.792696,0.984027,0.548100,0.400310,0,1,1
4,0.240706,0.913324,0.803689,0.212342,0.616651,9,1,2


### Dataset 2

In [3]:
data2.head()

,y,x1,x2,x3,x4,fx1,fx2,fx3
0,0.392433,6.004021,1.934715,-0.125241,0.874011,18,9,5
1,0.069365,9.814933,1.626457,-0.079265,0.180425,2,11,7
2,0.907241,4.761911,1.504155,-0.834101,0.096241,5,3,4
3,0.095627,6.935739,1.981092,-0.504128,0.912497,2,6,1
4,0.483698,12.768492,1.366475,-0.837983,0.342615,10,13,2


In [4]:
from pystata import summary_col
# some random combinations of fixed effects
fx_1 = {'Stock fixed effects': 'fx1','Year fixed effects': 'fx2'}
fx_2 = {'Stock fixed effects': 'fx1', 'Industry Fixed effects': 'fx3'}
fx_3 = {'Stock fixed effects': 'fx1','Year fixed effects': 'fx2', 'Industry Fixed effects': 'fx3'}

# Syntax: [data, regression specification, covariance type (enter cluster list),fixed effects]
reg_inputs = [[data1, 'y  ~ 1   + x1+ x2', 'robust', fx_1],
              [data1, 'y  ~ 1   + x1+ x2 ', 'robust', fx_1],
              [data2, 'y  ~ 1   + x1+ x2 + x3 + x4', 'robust',fx_2],
              [data2, 'y  ~ 1   + x1+ x2 + x3 ', 'robust',fx_2],
              [data1, 'y  ~ 1   + x1+ x2 + x4', ['fx1','fx2'],fx_2],
              [data2, 'y  ~ 1   + x1+ x2 + x3 + x4', ['fx1','fx2'],fx_3]
             ]

In [5]:
outputDir = '/home/shinc/Desktop/example/test/' # set the directory to save Stata output (log and results)
table = summary_col(reg_inputs) # read regression specification
table.set_dir(outputDir) # set the directory to save Stata output (log and results)
table.name = 'table_pystata' # set the name of the table
table.modelname = [ "Y1", "Y1","Var","Variable","Model name","Y",] # set the name for columns
table.order = ['x1', 'x2' , 'x3', 'x4'] # Determine independent variables order
table._main_() # transit data from python to Stata and write Stata do file accordingly
table.run_do() # run Stata do file

### Output 
1. the temporary data in input folder will be wiped out  (default option)
2. the log folder constains Stata log
3. the output folder constains 1. stata do script and 2. regression output (default format is tex)

In [6]:
! tree /home/shinc/Desktop/example/test/

/home/shinc/Desktop/example/test/
├── input
├── log
│   └── table_pystata.log
└── output
    ├── table_pystata.do
    └── table_pystata.tex

3 directories, 3 files


### Defaults output format is tex, however, it is hard to  show latex table in jupyter. Here, I provide a customized print function

In [7]:
table.print()

,Var,Y1 (1),Y1 (2),Var (3),Variable (4),Model name (5),Y (6)
0,x1,0.00602,0.00602,-0.000647,-0.000646,0.00568,-0.000620
1,,(0.19),(0.19),(-0.96),(-0.96),(0.18),(-0.92)
2,x2,-0.0169,-0.0169,0.00175,0.00177,-0.0125,0.00227
3,,(-0.53),(-0.53),(0.17),(0.18),(-0.34),(0.22)
4,x3,,,-0.00165,-0.00165,,-0.00174
5,,,,(-0.57),(-0.57),,(-0.63)
6,x4,,,-0.00132,,0.00549,-0.00185
7,,,,(-0.13),,(0.15),(-0.17)
8,---,---,---,---,---,---,---
0,Stock fixed effects,Yes,Yes,Yes,Yes,Yes,Yes


### Or simply save as html format (easy to print out in jupyter)

In [8]:
outputDir = '/home/shinc/Desktop/example/data/'
table = summary_col(reg_inputs)
table.set_dir(outputDir)
table.name = 'table_pystata'
table.modelname = [ "Y1", "Y1","Var","Variable","Model name","Y",]
table.order = ['x1', 'x2' , 'x3', 'x4']
table.outtype = 'html'
table._main_()
table.run_do()

In [9]:
table._readhtml_()